## Wikipedia Retrievers

In [1]:
from langchain_community.retrievers import WikipediaRetriever

In [4]:
# Initialize the retriever (optional: set language and top_k)
retriever = WikipediaRetriever(top_k_results=2, lang="en")

In [5]:
# Define your query
query = "The geopolitical history of india and pakistan from china perspective"

#Get relevant Wikipedia documents
docs = retriever.invoke(query)

In [6]:
for i, doc in enumerate(docs):
    print(f"\n-----Result {i+1} ----")
    print(f"Content:\n{doc.page_content}....") # Truncate for display


-----Result 1 ----
Content:
Pakistan and the United States established relations on 15 August 1947, a day after the independence of Pakistan, when the United States became one of the first nations to recognise the country.
The relationship between the two nations has been described as a "roller coaster" characterised by close coordination and lows marked by deep bilateral estrangement. Despite its troubled history, the Pakistani military once occupied an important place in American geopolitical strategy, and has been a major non-NATO ally since 2002. After Pakistan's participation in the Afghan peace process and the Taliban takeover in Afghanistan in 2021, a sizeable number of US policy makers are revisiting the United States' relations with Pakistan. At the same time, the strategic convergence of the United States and India has also brought greater pressure on Pakistani diplomacy.


== Background ==

During the Cold War (1945–1991), Pakistan allied itself with the Western Bloc led by

### Vector Store retriever

In [1]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

In [8]:
# Step 1: Your Source Documents
documents = [
    Document(page_content="LangChain helps developers build LLM Applications easily"),
    Document(page_content="Chroma is a vector database optimized for LLM-based search"),
    Document(page_content="Embeddings convert text into high-dimensional Vectors"),
    Document(page_content="OpenAI Provides powerful embedding models")
]

In [2]:
# Step 2: Initialize Embedding model
import os
os.environ['HF_HOME'] = "W:/huggingface_cache"
embedding_model = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B", model_kwargs={"trust_remote_code": True}, encode_kwargs={"normalize_embeddings":True})


m:\MyCompany\MachineLearning\GenAI\LangChain\mohan\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# Step 3: Create Chroma Vector Store in Memory
vectorstore = Chroma.from_documents(documents= documents, embedding=embedding_model, collection_name="mycollection")

# Step 4: Convert vector store into a retriever
retriever = vectorstore.as_retriever(search_kwargs = {"k":2})

#Step5: Invoke retriever
query = "What is chroma used for?"
results = retriever.invoke(query)



In [12]:
for i, doc in enumerate(results):
    print(f"\n----- Result {i+1} ----")
    print(doc.page_content)


----- Result 1 ----
Chroma is a vector database optimized for LLM-based search

----- Result 2 ----
LangChain helps developers build LLM Applications easily


In [13]:
results = vectorstore.similarity_search(query, k=2)

In [14]:
for i, doc in enumerate(results):
    print(f"\n----- Result {i+1} ----")
    print(doc.page_content)


----- Result 1 ----
Chroma is a vector database optimized for LLM-based search

----- Result 2 ----
LangChain helps developers build LLM Applications easily


When both retriever and similarity search giving same result, why retrievers?

- with retrievers, we can try out different search strategies which is not possible with vector store as it has only one search startegy.
- as retriever is a runnable object we can integrate that in chains

## MMR

In [16]:
from langchain.schema import Document
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM Based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more.")
        ]

In [17]:
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
import os
os.environ['HF_HOME'] = "W:/huggingface_cache"
embeddingmodel = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B", model_kwargs={"trust_remote_code": True}, encode_kwargs={"normalize_embeddings":True})

# Create a FAISS vector store from documents
vectorstore = FAISS.from_documents(documents=docs, embedding=embeddingmodel)

# Enable MMR in the retriever
retriever = vectorstore.as_retriever(search_type = "mmr",  # <-- this enables MMR)
                                     search_kwargs= {"k":3, "lambda_mult": 1}) # k = top results, Lambda_mult = relevance-diversity balance



In [18]:
Query = "What is LangChain?"
results = retriever.invoke(Query)

In [19]:
for i, doc in enumerate(results):
    print(f"\n ----- Results {i+1} ----")
    print(doc.page_content)

# with Less Lamba_Mult like 0.5 --> we get diverse results, with more Lamba_mult we get less diverse results

#When lambda_mult is 0.5, below is the output

"""
 ----- Results 1 ----
LangChain makes it easy to work with LLMs.

 ----- Results 2 ----
MMR helps you get diverse results when doing similarity search.

 ----- Results 3 ----
Embeddings are vector representations of text.
"""

# when lambda_mult is 1, below is the output



 ----- Results 1 ----
LangChain makes it easy to work with LLMs.

 ----- Results 2 ----
LangChain is used to build LLM Based applications.

 ----- Results 3 ----
LangChain supports Chroma, FAISS, Pinecone, and more.


'\n ----- Results 1 ----\nLangChain makes it easy to work with LLMs.\n\n ----- Results 2 ----\nMMR helps you get diverse results when doing similarity search.\n\n ----- Results 3 ----\nEmbeddings are vector representations of text.\n'

## Multi-Query Retriever

In [25]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
all_docs = [
Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}), 
Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}), 
Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}), 
Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}), 
Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}), 
Document(page_content="Python balances readbility with power, making it a popular system design language.", metadata={"source": "I2"}),
Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}), 
Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}), 
Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source":"I5"})
]

In [27]:
os.environ['HF_HOME'] = "W:/huggingface_cache"
embeddingmodel = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B", model_kwargs={"trust_remote_code": True}, encode_kwargs={"normalize_embeddings":True})

vectorstore = FAISS.from_documents(documents=all_docs, embedding = embeddingmodel)


In [28]:
#Create retriever
similarity_retriever = vectorstore.as_retriever(search_type = "similarity", search_kwargs = {"k" : 5})

In [29]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace

llm = HuggingFacePipeline.from_model_id(model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0", task="text-generation")
model = ChatHuggingFace(llm =llm)

Device set to use cpu


In [30]:
multiquery_retriever = MultiQueryRetriever.from_llm(llm = model, retriever= vectorstore.as_retriever(search_kwargs={"k":5}))

In [31]:
#Query 
query = "How to improve energy levels and maintain balance"

In [32]:
# Retriver results
similarity_results = similarity_retriever.invoke(query)
multiquery_results = multiquery_retriever.invoke(query)

In [34]:
print(len(multiquery_results))

7


In [33]:
for i, doc in enumerate(similarity_results):
    print(f"\n ------ Results {i+1} ---")
    print(doc.page_content)

print("*" * 80)

for i, doc in enumerate(multiquery_results):
    print(f"\n ------ Results {i+1} ---")
    print(doc.page_content)


 ------ Results 1 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

 ------ Results 2 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

 ------ Results 3 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

 ------ Results 4 ---
Regular walking boosts heart health and can reduce symptoms of depression.

 ------ Results 5 ---
Python balances reaility with power, making it a popular system design language.
********************************************************************************

 ------ Results 1 ---
Python balances reaility with power, making it a popular system design language.

 ------ Results 2 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

 ------ Results 3 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

 ------ Results 4 ---
Regular walking boosts heart health and can reduce symptoms of depressi

## ContextualCompressionRetriever

In [35]:
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [36]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [37]:
vectorstore = FAISS.from_documents(documents= docs, embedding= embeddingmodel)
baseretriever = vectorstore.as_retriever(search_kwargs = {"k":5})

In [38]:
# Setup the compressor using an LLM
compressor = LLMChainExtractor.from_llm(model)

In [39]:
# Create a contextual Compression retriever
compression_retriever = ContextualCompressionRetriever(base_retriever=baseretriever, base_compressor=compressor)

In [40]:
#query the retriever
query = "What is Photosynthesis"
compressed_results = compression_retriever.invoke(query)

In [41]:
for i, doc in enumerate(compressed_results):
    print(f"\n----- Results {i+1}----")
    print(doc.page_content)


----- Results 1----
<|user|>
Given the following question and context, extract any part of the context *AS IS* that is relevant to answer the question. If none of the context is relevant return NO_OUTPUT.

Remember, *DO NOT* edit the extracted parts of the context.

> Question: What is Photosynthesis
> Context:
>>>
The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years.
>>>
Extracted relevant parts:</s>
<|assistant|>
Remember, *DO NOT* edit the extracted parts of the context.

----- Results 2----
<|user|>
Given the following question and context, extract any part of the context *AS IS* that is relevant to answer the question. If none of the context is relevant return NO_OUTPUT.

Remember, *DO NOT* edit the extracted parts of the context.

> Question: What is Photosynthesis
> Context: